In [1]:
import tensorflow as tf
import os
import pickle
import numpy as np 
import warnings 

warnings.filterwarnings('ignore')
CIFAR_DIR = './data/cifar-10-batches-py/'
print(os.listdir(CIFAR_DIR))

D:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']


In [2]:
def load_data(filename):
    with open(filename, 'rb') as f:
        data = pickle.load(f, encoding='latin1')
        return data['data'], data['labels']

class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
        # 将所有的训练数据合并成一个大矩阵
        self._data = np.vstack(all_data)
        # 对图像数据进行归一化，因为有sigmod激活函数进行0，1之间的预测
        self._data = self._data / 127.5 - 1
        # 将所有的标签合并成一个向量
        self._labels = np.hstack(all_labels)
        self._need_shuffle = need_shuffle
        self._num_examples = self._data.shape[0]
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
            
    def _shuffle_data(self):
        n = np.random.permutation(self._num_examples)
        self._data = self._data[n]
        self._labels = self._labels[n]
        
    def next_batch(self, batch_size):
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception('have no more data')
        if end_indicator > self._num_examples:
            raise Exception('batch size is larger than all data')
        batch_data = self._data[self._indicator : end_indicator]
        batch_labels = self._labels[self._indicator : end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]
train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

batch_data, batch_labels = train_data.next_batch(10)
    

In [3]:
x = tf.placeholder(tf.float32, [None, 3072])
y = tf.placeholder(tf.int64, [None])

hidden1 = tf.layers.dense(x, 100, activation=tf.nn.relu)
hidden2 = tf.layers.dense(hidden1, 100, activation=tf.nn.relu)
hidden3 = tf.layers.dense(hidden2, 50, activation=tf.nn.relu)

y_ = tf.layers.dense(hidden3, 10)
loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)


predict = tf.argmax(y_, 1)
correct_prediction = tf.equal(predict, y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

In [4]:
init = tf.global_variables_initializer()
batch_size = 20 
train_steps = 10000
test_steps = 100

with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run([loss, accuracy, train_op], 
                                        feed_dict={x:batch_data, y:batch_labels})
        if (i + 1) % 500 == 0:
            print('[Train] step: %d, loss: %4.5f, acc: %4.5f' % (i + 1, loss_val, acc_val))
        if (i + 1) % 5000 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_data_batch, test_data_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run([accuracy], feed_dict={x:test_data_batch, y:test_data_labels})
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test] step: %d, acc: %4.5f' % (i + 1, test_acc))

[Train] step: 500, loss: 1.90225, acc: 0.45000
[Train] step: 1000, loss: 1.47771, acc: 0.50000
[Train] step: 1500, loss: 1.57430, acc: 0.40000
[Train] step: 2000, loss: 1.29158, acc: 0.55000
[Train] step: 2500, loss: 1.56753, acc: 0.35000
[Train] step: 3000, loss: 1.84847, acc: 0.35000
[Train] step: 3500, loss: 1.73265, acc: 0.45000
[Train] step: 4000, loss: 1.36447, acc: 0.45000
[Train] step: 4500, loss: 1.92240, acc: 0.45000
[Train] step: 5000, loss: 1.20861, acc: 0.60000
[Test] step: 5000, acc: 0.46900
[Train] step: 5500, loss: 1.37267, acc: 0.55000
[Train] step: 6000, loss: 1.81779, acc: 0.30000
[Train] step: 6500, loss: 1.29555, acc: 0.45000
[Train] step: 7000, loss: 1.55933, acc: 0.55000
[Train] step: 7500, loss: 1.54335, acc: 0.55000
[Train] step: 8000, loss: 2.03859, acc: 0.40000
[Train] step: 8500, loss: 1.11400, acc: 0.55000
[Train] step: 9000, loss: 1.42376, acc: 0.45000
[Train] step: 9500, loss: 1.18972, acc: 0.50000
[Train] step: 10000, loss: 1.17657, acc: 0.60000
[Test] s